In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [20]:
filename = 'input2.txt'
input_text = au.read_txt_file_lines(filename)
n_rows = len(input_text)

## coord min/max, both inclusive
if filename == 'input.txt':
    coord_min = 200000000000000
    coord_max = 400000000000000
elif filename == 'input2.txt':
    coord_min = 7 
    coord_max = 27

In [27]:
class Hailstone:
    def __init__(self, input_line, coord_min, coord_max):
        self.process_text(input_line)
        self.solve_linear_xy()
        self.coord_min = coord_min
        self.coord_max = coord_max        

    def process_text(self, input_line):
        pos, vel = input_line.split('@')
        pos = np.array([int(x.strip()) for x in pos.split(',')])
        vel = np.array([int(x.strip()) for x in vel.split(',')])
        self.pos = pos
        self.vel = vel

        self.px = pos[0]
        self.py = pos[1]
        self.pz = pos[2]
        self.vx = vel[0]
        self.vy = vel[1]
        self.vz = vel[2]

        # assert self.px >= coord_min and self.px <= coord_max
        # assert self.py >= coord_min and self.py <= coord_max
        # assert self.pz >= coord_min and self.pz <= coord_max

    def solve_linear_xy(self):
        '''get solution to y = ax + b'''
        self.slope = self.vy / self.vx
        self.intercept = self.py - self.slope * self.px

    def get_xy_end_point(self):
        x = self.coord_max
        y = self.slope * x + self.intercept
        return (x, y)
    
    def __repr__(self):
        return f'pos: {self.pos}. vel: {self.vel}'
    
def calculate_intercept_two_lines(h1, h2):
    '''return x,y of intercept of two lines'''
    if h1.slope == h2.slope:
        return None, None 
    x = (h2.intercept - h1.intercept) / (h1.slope - h2.slope)
    y = h1.slope * x + h1.intercept
    return (x, y)

def determine_intercept_in_bounds(h1, h2):
    '''determine if intercept is in bounds'''
    x, y = calculate_intercept_two_lines(h1, h2)
    if x is None:
        return False
    if x >= h1.coord_min and x <= h1.coord_max and y >= h1.coord_min and y <= h1.coord_max:
        pass
    else:
        return False
    
    if h1.vx > 0 and x < h1.px:
        return False
    elif h1.vx < 0 and x > h1.px:
        return False
    elif h1.vy > 0 and y < h1.py:
        return False
    elif h1.vy < 0 and y > h1.py:
        return False
    if h2.vx > 0 and x < h2.px:
        return False
    elif h2.vx < 0 and x > h2.px:
        return False
    elif h2.vy > 0 and y < h2.py:
        return False
    elif h2.vy < 0 and y > h2.py:
        return False
    return True

list_hailstones = [Hailstone(x, coord_min, coord_max) for x in input_text]

In [28]:
n_valid_intercepts = 0
for ii in tqdm(range(len(list_hailstones))):
    for jj in range(ii + 1, len(list_hailstones)):
        h1 = list_hailstones[ii]
        h2 = list_hailstones[jj]
        if determine_intercept_in_bounds(h1, h2):
            n_valid_intercepts += 1

print(n_valid_intercepts)

100%|██████████| 5/5 [00:00<00:00, 974.74it/s]

2


## part 2:
Thoughts:
- First thought I could do something with border (maybe a/some lines are only a few time steps away, thereby revealing a v small subset of initial positions), but actually i don't think the border constraints is present in part 2. 
- New idea: the time component at first seemed to make things extra difficult, but actually, in 3D (NOT 2D), there should only be one intersecting line I expect? Because if multiple different 3D-slopes are on the same line, then at the next pos they won't be. So actually, we probably just need to find any line that connects all lines regardles of time (and I'm sure the time will work out). 
- For 2 lines, there are infinite possibilities
- For 3 lines, I think if there is 1 possibliity, then you can rotate (wiht middle line at centre point) it so that there infinite 
- for 4, no longer? -> could I start with any 4 lines, find intersection and then see if that works for others..? 

.. 

Try to solve analytically:
For hail `k`, its line is given by $y = a_k * x + b_k$. The solution is of the form $y = a_S * x + b_S$. So that means that for all $k$, there is a $(x, y)$ that satifies both lines. 


In [71]:
import scipy.optimize

def optimise_four_lines(h1, h2, h3, h4):

    init_guess = [0, 1]
    tmp_fun = lambda x: loss_fun_intersect(h1, h2, h3, h4, x[0], x[1]) 
    result = scipy.optimize.minimize(tmp_fun, init_guess, method='L-BFGS-B')

    if result.success:
        pos_line, vel_line = line_from_two_crossings(h1, h2, result.x[0], result.x[1])
        assert np.all(np.isclose(pos_line + vel_line * result.x[0], h1.pos + h1.vel * result.x[0]))
        assert np.all(np.isclose(pos_line + vel_line * result.x[1], h2.pos + h2.vel * result.x[1]))
    return result

def line_from_two_crossings(h1, h2, t_a, t_b):
    pos_a = h1.pos + t_a * h1.vel 
    pos_b = h2.pos + t_b * h2.vel
    delta_t = t_b - t_a 
    if delta_t == 0:
        print('meh')
    vel_line = (pos_a - pos_b) / delta_t 
    pos_line = pos_a - t_a * vel_line
    return pos_line, vel_line

def loss_fun_intersect(h1, h2, h3, h4, t_a, t_b):
    pos_line, vel_line = line_from_two_crossings(h1, h2, t_a, t_b)
    loss_3 = min_distance_two_lines(pos1=pos_line, pos2=h3.pos, vel1=vel_line, vel2=h3.vel)
    loss_4 = min_distance_two_lines(pos1=pos_line, pos2=h4.pos, vel1=vel_line, vel2=h4.vel)
    return loss_3 + loss_4

def distance_two_lines(pos1, pos2, vel1, vel2, t):
    pos1_current = pos1 + t * vel1 
    pos2_current = pos2 + t * vel2 
    dist = np.sqrt(np.sum(np.power(pos1_current - pos2_current, 2)))
    return dist 

def min_distance_two_lines(pos1, pos2, vel1, vel2):
    min_dist = scipy.optimize.minimize_scalar(lambda x: distance_two_lines(pos1=pos1, pos2=pos2, vel1=vel1,
                                                                           vel2=vel2, t=x))
    assert min_dist.success
    return min_dist.x

In [72]:
# min_distance_two_lines(list_hailstones[0].pos, list_hailstones[1].pos, list_hailstones[0].vel, list_hailstones[1].vel)
result = optimise_four_lines(list_hailstones[0], list_hailstones[1], list_hailstones[2], list_hailstones[3])
print(result)

AssertionError: 

In [63]:
line_from_two_crossings(list_hailstones[0], list_hailstones[1], result.x[0], result.x[1])

(array([53.33940632, -3.76822095, 71.98996058]),
 array([ 1.76809677, -0.83999335,  2.6075996 ]))